In [1]:
import os
from cralds_base.dataio import read_file
from cralds.dataio.fileio.read.specific.trios_rheo_txt import TriosRheoReader
import sys
sys.path.append(r'../src/')
from hermes_rheo.transforms.rheo_analysis import RheoAnalysis 
from hermes_rheo.transforms.automated_mastercurve import AutomatedMasterCurve
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")

In [2]:
filepath = r'Q:\Projects\Rheology_OP\CRAL_hermes\damien_vadillo\SoR-2024\468MP\153 - hermes -  468mp - 0.3-60 rad_s 10pct.2024.09.16.txt' # Chirp data
experiment = read_file(filepath, create_composite_datasets=True)
pipeline = RheoAnalysis() 

processed_experiment = pipeline(experiment)
print(processed_experiment)

ExperimentSet
-------------

Summary
-------
Number of Experiments: 20

Varied Conditions
-----------------
method

index	n_rep	method	Data Types	Data Types	Data Types	
0	1	Step name	Arbitrary Wave - 1	storage modulus(Pa)=f(angular frequency[rad/s]) (156 points)	velocity(rad/s)=f(torque[g.cm]) (7499 points)	strain filtered masked(a.u)=f(time masked[s]) (7000 points)	
1	1	Step name	Arbitrary Wave - 2	storage modulus(Pa)=f(angular frequency[rad/s]) (156 points)	velocity(rad/s)=f(torque[g.cm]) (7499 points)	strain filtered masked(a.u)=f(time masked[s]) (7000 points)	
2	1	Step name	Arbitrary Wave - 3	storage modulus(Pa)=f(angular frequency[rad/s]) (156 points)	velocity(rad/s)=f(torque[g.cm]) (7499 points)	strain filtered masked(a.u)=f(time masked[s]) (7000 points)	
3	1	Step name	Arbitrary Wave - 4	storage modulus(Pa)=f(angular frequency[rad/s]) (156 points)	velocity(rad/s)=f(torque[g.cm]) (7499 points)	strain filtered masked(a.u)=f(time masked[s]) (7000 points)	
4	1	Step name	Arbitrary Wav

In [3]:
import numpy as np
import copy

# Initialize lists to store the extracted data
angular_frequencies = []
storage_moduli = []
loss_moduli = []
complex_moduli = []
complex_viscosities = []
phase_angles = []
step_times = []
strains = []
stresses = []
temperatures = []

# Number of steps 
steps = len(processed_experiment)

# Process each experiment to extract data
for step in range(steps):
    dataset_frequency = processed_experiment[step][0].datasets[0]

    angular_frequency = copy.deepcopy(dataset_frequency.x_values)
    storage_modulus = copy.deepcopy(dataset_frequency.y_values)

    dataset_frequency.switch_coordinates('loss modulus', 'complex modulus')
    loss_modulus = copy.deepcopy(dataset_frequency.x_values)
    complex_modulus = copy.deepcopy(dataset_frequency.y_values)

    dataset_frequency.switch_coordinates('complex viscosity', 'tan(delta)')
    complex_viscosity = copy.deepcopy(dataset_frequency.x_values)
    phase_angle = copy.deepcopy(dataset_frequency.y_values)

    angular_frequencies.append(angular_frequency)
    storage_moduli.append(storage_modulus)
    loss_moduli.append(loss_modulus)
    complex_moduli.append(complex_modulus)
    complex_viscosities.append(complex_viscosity)
    phase_angles.append(phase_angle)

    dataset_time = processed_experiment[step][0].datasets[1]
    dataset_time.switch_coordinates('time_cut', 'strain filtered')
    step_time = copy.deepcopy(dataset_time.x_values)
    strain = copy.deepcopy(dataset_time.y_values)

    dataset_time.switch_coordinates('temperature', 'stress filtered')
    temperature = copy.deepcopy(dataset_time.x_values)
    stress = copy.deepcopy(dataset_time.y_values)

    step_times.append(step_time)
    strains.append(strain)
    temperatures.append(temperature)
    stresses.append(stress)


In [42]:
import numpy as np

# Read the strain and time data from the text file
chirp_data = np.loadtxt('chirp_waveform_data.txt', delimiter='\t', skiprows=1)
chirp_time = chirp_data[:, 0]  # Time column from the file
chirp_strain = chirp_data[:, 1]  # Strain column from the file

# Create a directory to save the images if it doesn't exist
output_dir = "output_plots"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Plot data for each experiment and save as PNG
for step in range(1):
    f = angular_frequencies[step]
    stor = storage_moduli[step]
    loss = loss_moduli[step]
    complex_mod = complex_moduli[step]  # Assuming complex_moduli is prepared similarly to stor and loss
    time = step_times[step]
    strain = strains[step]
    stress = stresses[step]
    average_temperature = np.average(temperatures[step])
    title = f"Arbitrary Wave {step + 1}, Temperature = {average_temperature:.2f} °C"

    # Create a custom GridSpec layout
    fig = plt.figure(figsize=(12, 5))
    gs = gridspec.GridSpec(2, 2, width_ratios=[2, 1], height_ratios=[2, 2])

    # Moduli plot (left)
    ax1 = plt.subplot(gs[:, 0])
    ax1.plot(f, stor, marker='s', label=r"$E'$", color='red', markersize=10, linestyle='-')
    ax1.plot(f, loss, marker='^', label=r"$E''$", color='blue', markersize=10, markerfacecolor='none', linestyle='--')
    ax1.set_title(title, fontsize=15)
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlabel(r'$\omega$ (rad/s)', fontsize=20)
    ax1.set_ylabel(r"$G'$, $G''$ (Pa)", fontsize=20)
    ax1.legend(loc='best', frameon=False, prop={'size': 15}, ncol=1, fontsize=15)
    ax1.set_ylim(1e3, 5e8)  # Setting y-axis limits for moduli

    # Time vs strain plot (top right)
    ax3 = plt.subplot(gs[0, 1])
    ax3.plot(time, strain, linewidth=2, color='seagreen', label='Measured Strain')
    
    # Add the strain from the text file with a dashed black line
    ax3.plot(chirp_time, chirp_strain, 'k-', linewidth=1,  zorder=-1, label='Input Strain')
    ax3.legend(fontsize=6, frameon=False, loc=2)

    
    ax3.set_xlabel('Time (s)', fontsize=16)
    ax3.set_ylabel('Strain', fontsize=16)
    ax3.axhline(0, color='black', linestyle='--', linewidth=1)  # Dashed line at y=0

    # Time vs stress plot (bottom right)
    ax4 = plt.subplot(gs[1, 1])
    ax4.plot(time, stress, linewidth=2, color='darkorange')
    ax4.set_xlabel('Time (s)', fontsize=16)
    ax4.set_ylabel('Stress (Pa)', fontsize=16)
    ax4.axhline(0, color='black', linestyle='--', linewidth=1)  # Dashed line at y=0

    plt.tight_layout()

    # Save the plot as a PNG file
    plot_filename = os.path.join(output_dir, f"plot_{step + 1}.png")
    plt.savefig(plot_filename, dpi=300)
    plt.close()  # Close the figure to avoid displaying it


In [5]:
import cv2
import os

# Directory containing the saved PNG plots
output_dir = "output_plots"

# Get the list of image files and sort them by step number
image_files = sorted([img for img in os.listdir(output_dir) if img.endswith(".png")], key=lambda x: int(x.split('_')[1].split('.')[0]))

# Define the codec and create a VideoWriter object
frame = cv2.imread(os.path.join(output_dir, image_files[0]))
height, width, layers = frame.shape
video_filename = 'plot_video.avi'
fps = 2  # 2 frames per second to speed up the video
video = cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

# Write each image to the video in sorted order
for image in image_files:
    img_path = os.path.join(output_dir, image)
    frame = cv2.imread(img_path)
    video.write(frame)

# Release the video writer object
video.release()

print(f"Video saved as {video_filename}")


Video saved as plot_video.avi


In [26]:
print(processed_experiment[0][0].details['strain_applied'])
print(processed_experiment[0][0].details['waiting_time'])
print(processed_experiment[0][0].details['taping_parameter'])
print(processed_experiment[0][0].details['initial_frequency'])
print(processed_experiment[0][0].details['final_frequency'])
print(processed_experiment[0][0].details['oscillation_period'])
print(processed_experiment[0][0].details['sampling_frequency'])

8.5e-05
1.0
0.1
0.3
60.041
13.999999999999998
500
